# Tutorial 2.2: V$_{phys}$ Parametrization

In this section, we show how the $V_{phys}$ part of MB-nrg PEFs are parametrized. $V_{phys}$ is the underlying "physically-inspired" baseline consisting of dispersion and electrostatics interactions upon which the many-body polynomials are applied as "corrections". As a reminder, $V_{phys}$ has the form:

#### <center> $V_{phys} = V_{disp} + V_{elec}$ </center>

Here, $V_{disp}$ is the disperison energy and $V_{elec}$ is the electrostatics energy. $V_{disp}$ has the form:

#### <center> $    V_{\mathrm{disp}} = 
    \sum_{{i<j}}^{N}
    -\Delta_{ij}f(\mathrm{d}_{6, ij},R_{ij}) \frac{C_{6,ij}}{R^6_{ij}}$ </center>

where, $R_{ij}$, $C_{6, ij}$, and $d_{6, ij}$ are the distance between atoms, dispersion coefficient, and damping parameter for atom pair $i$, $j$. $f() is the Tang-Toennies damping function of the form:

#### <center> $	f(\mathrm{d}_{6, ij},R_{ij}) = 1 - \mathrm{exp}(-\mathrm{b}_{ij}R_{ij})\sum_{n=0}^6\frac{(\mathrm{d}_{6, ij}R_{ij})^n}{n!}
$ </center>

And $V_{elec}$ has the form:

#### <center> $V_{elec} = V_{qq} + V_{q\mu} + V_{\mu\mu} + V_{pol}$ </center>

where these terms are the charge-charge, charge-dipole, dipole-dipole, and polarization energi, respectively.

Entering the expressions for each term in $V_{elec}$ are the atomic charges ($q_i$) polarizabilities ($p_i$), polarizability factors ($\alpha_i$) and screening parameters.

Before we can begin constructing our MB-nrg surface, we must calculate all the parameters that enter $V_{phys}$

## 2.2.0 Definitions and Imports

Please execute the imports and paths notebooks:

In [ ]:
%run Tutorial_2_Imports.ipynb

In [ ]:
%run Tutorial_2_Paths.ipynb

## 2.2.1 Polarizabilities and Polarizability Factors.

All MB-nrg models set the polarizability factors equal to the polarizabilities and calculate them in the following way:


#### <center> $p_i = \alpha_i = p^{free}_i  \frac{v_i}{v^{free}_i} ^ {4/3}$ </center>

Here, $p^{free}_i$, $v^{free}_i$, and $v_i$ are the free polarizability, free volume, and effective volume for atom $i$. We calculate the free volumes and effective volumes using QChem's implementation of the eXchange-hole Dipole Moment (XDM) model, and the free polarizabilities have been tabulated at the CCSD(T) level.

We will use the optimized CH$_3$-NH$_2$ structure from the previous notebook as the input structure for XDM, so lets read that in:

In [ ]:
definition_CH3NH2 = read_definition(paths.definitions.systems.CH3NH2)
minimized_structure_CH3NH2 = read_system(definition_CH3NH2, paths.structures.minimized_CH3NH2)

To run the XDM calculation to calculate the polarizabilities, do the following:

In [ ]:
polarizabilities_CH3NH2 = calculate_polarizabilities_by_XDM(
        minimized_structure_CH3NH2,
        logs_directory=f"{paths.logs}",
        restart_path=paths.restart.polarizabilities_CH3NH2,
        method="wb97m-v",
        basis="aug-cc-pVTZ",
        num_threads=16,
        mem_mb=32000
)

As is the case for MB-pol and all MB-nrg PEFs, we set the polarizability factors equal to the polarizabilities:

In [ ]:
polarizability_factors_CH3NH2 = polarizabilities_CH3NH2

Lets take a look at our polarizabilities (units $Å^3$):

In [ ]:
print(polarizabilities_CH3NH2)
print(polarizability_factors_CH3NH2)

For water, we will use the polarizabilities of MB-pol. This is required if we want our new PEF to be "compatible" with MB-pol:

In [ ]:
polarizabilities_H2O = {
    "E": 1.310,
    "F": 0.294
}
polarizability_factors_H2O = {
    "E": 1.310,
    "F": 0.294
}

Lets save the polarizabilities to a file:

In [ ]:
write_parameters(paths.Vphys.polarizabilities, {**polarizabilities_CH3NH2, **polarizabilities_H2O})
write_parameters(paths.Vphys.polarizability_factors, {**polarizability_factors_CH3NH2, **polarizability_factors_H2O})

## 2.2.2 Atomic Radii

Next we will calculate the atomic radii. While not directly needed for the PEF, the atomic radii will be used in the calculation of the dispersion damping factors, and to choose the grid points that we will use to parametrize the charges.

We also use XDM to calculate the radii. XDM gives us an effective volume ($v_i$) for each atom from which we can compute a radius ($r_i$), assuming that the volume is a perfect sphere:

#### <center> $ r_i = \frac{3 v_i}{4 \pi}^\frac{1}{3} $ </center>

In [ ]:
radii_CH3NH2 = calculate_radii_by_XDM(
        minimized_structure_CH3NH2,
        logs_directory=f"{paths.logs}",
        restart_path=paths.restart.radii_CH3NH2,
        method="wb97m-v",
        basis="aug-cc-pVTZ",
        num_threads=16,
        mem_mb=32000
)

Here are our radii (units $Å$):

In [ ]:
radii_CH3NH2

We will also need radii for water to calculate the damping factors for the methylamine - water interaction.

Read in the optimized water structure from the previous notebook:

In [ ]:
definition_H2O = read_definition(paths.definitions.systems.H2O)
minimized_structure_H2O = read_system(definition_H2O, paths.structures.minimized_H2O)

And calculate the radii:

In [ ]:
radii_H2O = calculate_radii_by_XDM(
        minimized_structure_H2O,
        logs_directory=f"{paths.logs}",
        restart_path=paths.restart.radii_H2O,
        method="wb97m-v",
        basis="aug-cc-pVTZ",
        num_threads=16,
        mem_mb=32000,
)

In [ ]:
radii_H2O

Save them to a file:

In [ ]:
write_parameters(paths.Vphys.radii, {**radii_CH3NH2, **radii_H2O})

## 2.2.3 Atomic Partial Charges.

Previous MB-nrg models have used a variety of methods to calculate the atomic partial charges. MB-pol uses the Partridge-Schwenke geometry-dependent atomic partial charges. Other models have used either Charge Model 5 (cite) or CHELPG (cite). The most recent generation of models has used charges fit to reproduce a monomer's multipole moments using an approach similar to the Stewart fitting appraoch. However, we have recently developed a new apprach based on CHELPG that we feel provides the most useful model of a monomer's charge for our purposes. 

We will fit the partial charges of methylamine to reproduce the electrostatic potential on a number of grid points. However unlike CHELPG, which uses grid points very close to the monomer's electron density (usually within 4 $Å$ of the nucleii), we will fit to points quite far away from the electron density. The justification for this is that since we will have the many-body polynomials to correct at short-range, so it is most important that our electrostatics model reproduces long-range interactions. We will fit to grid points that are between 9 and 11 $Å$ past the edge of the electron density of our monomer.

We will fit the charges by minimizing the squared error in the electrostatic potential at each grid point with respect to $q_i$:

#### <center> $ \sum_{n = 1}^N \sum_{j=1}^{J_n} (P_{ref, n}(j) - P_{pred, n}(j, q_i))^2 $ </center>

where $P_{ref, n}(j)$ gives the reference ESP at the jth grid point of geometry n, and $P_{pred, n}(j, q_i)$ gives the predicted ESP at the same site.

Before we can fit, we need to generate the $N$ configurations whose ESP we will fit on. We can do this using MB-Fit's normal modes configuration generator. First, read in the normal modes for the optimized methylamine geometry we calculated earlier:

In [ ]:
vibrational_modes_CH3NH2 = read_vibrational_modes(paths.structures.vibrational_modes_CH3NH2)

Then, lets get 25 geometries from the normal modes configuration generator:

In [ ]:
configuration_generator = NormalModesConfigurationGenerator(
        minimized_structure_CH3NH2,
        vibrational_modes_CH3NH2,
        classical=False,
        distribution="constant",
        temperature=1000
)
charge_parametrization_configurations_CH3NH2 = list(configuration_generator.generate_configurations(num_configs=25, seed=12345))
write_systems(paths.Vphys.charge_configs_CH3NH2, charge_parametrization_configurations_CH3NH2)

As we can see, the normal modes configuration generator has produced a bunch of geometries around the minimum structure:

In [ ]:
render_overlayed_systems(
    systems=charge_parametrization_configurations_CH3NH2,
    centerer=lambda system: system.atoms[0].point,
    aligner=lambda system: (system.atoms[4].point, system.atoms[3].point),
    alpha=0.05
)

First, we calculate the grid points that we will fit the ESP on. Here, we take grid points that are between 9.0 + atomic_radius and 11.0 + atomic_radius distance from the nucleii:

In [ ]:
grid_points, grid_point_weights = get_ESP_grid_points(
        charge_parametrization_configurations_CH3NH2,
        atomic_radii=radii_CH3NH2,
        grid_radii=[9.0, 10.0, 11.0]
)

Now, we calculate the ESP at the grid points using electronic structure theory. We usually use wB97M-V/aug-cc-pVTZ, since this functional provides a good description of the density:

In [ ]:
esp_at_grid_points = calculate_ESP_at_grid_points(
        charge_parametrization_configurations_CH3NH2,
        grid_points,
        esp_scratch_dir=paths.scratch.esp_CH3NH2,
        restart_path=paths.restart.esp_CH3NH2,
        method="wB97M-V",
        basis="aug-cc-pVTZ",
        num_threads=16,
        mem_mb=32000
)

Finally, fit the charges to reproduce the ESP:

In [ ]:
charges_CH3NH2 = fit_charges_to_ESP(
        definition_CH3NH2,
        charge_parametrization_configurations_CH3NH2,
        grid_points=grid_points,
        grid_point_weights=grid_point_weights,
        grid_point_reference_values=esp_at_grid_points,
        restart_path=paths.restart.charge_fitting_CH3NH2,
        formal_charge=0.0,
    
)

In [ ]:
charges_CH3NH2

Note that these charges may not be what one would predict - for example, its a bit strange to see a + charge on carbon (symmetry 'A'). However, we are interested less in accurately modelling the electron density and more in predicting long-range electrostatic interactions, so this is OK.

Save the parameters to a file:

In [ ]:
write_parameters(paths.Vphys.charges, charges_CH3NH2)

## 2.2.4 Dispersion Parameters

Next, we will calculate the dispersion coefficients ($C_{6, ij}$) and damping parameters ($d^6_{ij}$).

The $C_{6, ij}$ are computed directly by XDM, see [QChem's documentation about the XDM module](https://manual.q-chem.com/5.4/sect_XDM.html) for more info about how.

To run the XDM calculation for the methylamine-water C6, do:

In [ ]:
C6_CH3NH2_H2O = calculate_C6_by_XDM(
        minimized_structure_CH3NH2,
        minimized_structure_H2O,
        paths.logs,
        paths.restart.dispersion_CH3NH2_H2O,
        method="wb97m-v",
        basis="aug-cc-pVTZ",
        num_threads=16,
        mem_mb=32000
)

Which produces the C6 parameters (untis $kcal / (mol \cdot A^6)$):

In [ ]:
C6_CH3NH2_H2O

We will also need to calcualte the CH3NH2 -- CH3NH2 dispersion parameters. While we are only seeking to model a single CH3NH2 in water for now, we still need to calculate the self-interaciton C6 to get the "long-range" C6 used by the reciprocal space part of the dispersion.

In [ ]:
C6_CH3NH2_CH3NH2 = calculate_C6_by_XDM(
        minimized_structure_CH3NH2,
        minimized_structure_CH3NH2,
        paths.logs,
        paths.restart.dispersion_CH3NH2_CH3NH2,
        method="wb97m-v",
        basis="aug-cc-pVTZ",
        num_threads=16,
        mem_mb=32000
)

Which produces:

In [ ]:
C6_CH3NH2_CH3NH2

We will take the water -- water C6 from MB-pol:

In [ ]:
C6_H2O_H2O = {
    ("E", "E"): 2.373212214147944e+02,
    ("E", "F"): 8.349556669872743e+01,
    ("F", "F"): 2.009358600184719e+01,
}

Save the C6 parameters to a file:

In [ ]:
write_parameters(paths.Vphys.C6, {**C6_CH3NH2_H2O, **C6_CH3NH2_CH3NH2, **C6_H2O_H2O})

Now we move on to the disperison damping parameters $d_{6, ij}$

The damping parameters are chosen so that the dispersion is exactly half on at a distance equal to the sum of the two atom's atomic radii ($r_i + r_j$). This is done by solving the following equation for ${d}_{6, ij}$:

#### <center> $ \frac{1}{2} = f({d}_{6, ij}, r_i + r_j) $ </center>

We calculated the radii earlier, call the following function to solve the above equation for ${d}_{6, ij}$:

In [ ]:
d6_CH3NH2_H2O = calculate_d6_from_radii(
        radii1=radii_CH3NH2,
        radii2=radii_H2O,
        damping_at_radii_sum=0.5
)

Here are the calculated damping parameters (units $A^{-1}$):

In [ ]:
d6_CH3NH2_H2O

And again for the CH3NH2 -- CH3NH2 damping parameters:

In [ ]:
d6_CH3NH2_CH3NH2 = calculate_d6_from_radii(
        radii1=radii_CH3NH2,
        radii2=radii_CH3NH2,
        damping_at_radii_sum=0.5
)

In [ ]:
d6_CH3NH2_CH3NH2

We take the water -- water d6 from MB-pol

In [ ]:
d6_H2O_H2O = {
    ("E", "E"): 9.295485815062264e+00,
    ("E", "F"): 9.775202425217957e+00,
    ("F", "F"): 9.406475169954112e+00,
}

In [ ]:
write_parameters(paths.Vphys.d6, {**d6_CH3NH2_H2O, **d6_CH3NH2_CH3NH2, **d6_H2O_H2O})

## 2.2.5 Screening Parameters

The last peice of $V_{phys}$ is the electrostatic screening parameters. MB-nrg PEFs screen the electrostatics at very short range using the Tang-Toennies screening functions.

The screening for the charge-charge and charge-dipole interactions is always 0.4 (unitless):

In [ ]:
aCC = 0.4

In [ ]:
aCD = 0.4

Additionally, the screening for dipole-dipole non-bonded and 1-4 interactions is always 0.055:

In [ ]:
aDD = 0.055
aDD14 = 0.055

The dipole-dipole screening parameters for 1-2 and 1-3 interactions are a bit more complicated. For CH3NH2 -- CH3NH2 and CH3NH2 -- water interactions we use 0.3, and for water -- water interactions we use 0.626 for the 1-2 and 0.055 for the 1-3 interactions. This is to maintain consistence with MB-pol.

In [ ]:

all_symmetries = ["A", "B", "C", "D", "E", "F"]

aDD12 = {tuple(sorted([symmetry1, symmetry2])): 0.3 for symmetry1, symmetry2 in itertools.combinations_with_replacement(all_symmetries, r=2)}
aDD12[("E", "E")] = 0.626
aDD12[("E", "F")] = 0.626
aDD12[("F", "F")] = 0.626

aDD13 = {tuple(sorted([symmetry1, symmetry2])): 0.3 for symmetry1, symmetry2 in itertools.combinations_with_replacement(all_symmetries, r=2)}
aDD13[("E", "E")] = 0.055
aDD13[("E", "F")] = 0.055
aDD13[("F", "F")] = 0.055

## 2.2.6 Validation of $V_{phys}$ Parametrization

Great, we have our $V_{phys}$ parameters. To get a sense of the baseline we will add our many-body PIPs to, lets compare it to the electronic structure calculations we did earlier.

Since our monomers are small enough that everything is excluded, there is no $V_{phys}$ contribution at the 1-body level, so there is nothing interesting to see there. We will jump straight to the 2-body.

In [ ]:
definition_CH3NH2_H2O = read_definition(paths.definitions.systems.CH3NH2_H2O)

Lets read in the first 2-body scan and calcualte the energy contribution from $V_{phys}$:

In [ ]:
scan_CH3NH2_H2O_1, scan_CH3NH2_H2O_1_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.scans.CH3NH2_H2O.first_energies)
scan_CH3NH2_H2O_1_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_1, scan_CH3NH2_H2O_1_energies)

scan_CH3NH2_H2O_1_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_CH3NH2_H2O,
    scan_CH3NH2_H2O_1,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6=C6_CH3NH2_H2O,
    symmetries_to_d6=d6_CH3NH2_H2O,
    mbpol_monomers=[False, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

scan_CH3NH2_H2O_1_Vphys_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_1, scan_CH3NH2_H2O_1_Vphys_energies)

And then plot:

In [ ]:
plot_scan(
        scan_CH3NH2_H2O_1,
        {
                "MP2/aug-cc-pVTZ": scan_CH3NH2_H2O_1_2b_energies,
                "$V_{phys}$": scan_CH3NH2_H2O_1_Vphys_2b_energies,
        },
        4,
        7,
        title="2-body Energies",
        ymin=-4,
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[7].point, system.atoms[0].point)
)

And similarly for the other two 2-body scans:

In [ ]:
scan_CH3NH2_H2O_2, scan_CH3NH2_H2O_2_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.scans.CH3NH2_H2O.second_energies)
scan_CH3NH2_H2O_2_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_2, scan_CH3NH2_H2O_2_energies)

scan_CH3NH2_H2O_2_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_CH3NH2_H2O,
    scan_CH3NH2_H2O_2,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6=C6_CH3NH2_H2O,
    symmetries_to_d6=d6_CH3NH2_H2O,
    mbpol_monomers=[False, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

scan_CH3NH2_H2O_2_Vphys_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_2, scan_CH3NH2_H2O_2_Vphys_energies)

In [ ]:
plot_scan(
        scan_CH3NH2_H2O_2,
        {
                "MP2/aug-cc-pVTZ": scan_CH3NH2_H2O_2_2b_energies,
                "$V_{phys}$": scan_CH3NH2_H2O_2_Vphys_2b_energies,
        },
        4,
        7,
        title="2-body Energies",
        ymin=-8,
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[7].point, system.atoms[0].point)
)

In [ ]:
scan_CH3NH2_H2O_3, scan_CH3NH2_H2O_3_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.scans.CH3NH2_H2O.third_energies)
scan_CH3NH2_H2O_3_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_3, scan_CH3NH2_H2O_3_energies)

scan_CH3NH2_H2O_3_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_CH3NH2_H2O,
    scan_CH3NH2_H2O_3,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6=C6_CH3NH2_H2O,
    symmetries_to_d6=d6_CH3NH2_H2O,
    mbpol_monomers=[False, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

scan_CH3NH2_H2O_3_Vphys_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_3, scan_CH3NH2_H2O_3_Vphys_energies)

In [ ]:
plot_scan(
        scan_CH3NH2_H2O_3,
        {
                "MP2/aug-cc-pVTZ": scan_CH3NH2_H2O_3_2b_energies,
                "$V_{phys}$": scan_CH3NH2_H2O_3_Vphys_2b_energies,
        },
        0,
        7,
        title="2-body Energies",
        ymin=-2,
        ymax=2,
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[7].point, system.atoms[4].point)
)

Now, lets do the same for the 3-body scans:

In [ ]:
definition_CH3NH2_H2O_H2O = read_definition(paths.definitions.systems.CH3NH2_H2O_H2O)

In [ ]:
scan_CH3NH2_H2O_H2O_1, scan_CH3NH2_H2O_H2O_1_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O_H2O, paths.scans.CH3NH2_H2O_H2O.first_energies)
scan_CH3NH2_H2O_H2O_1_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_H2O_1, scan_CH3NH2_H2O_H2O_1_energies)

scan_CH3NH2_H2O_H2O_1_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_CH3NH2_H2O_H2O,
    scan_CH3NH2_H2O_H2O_1,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6={**C6_CH3NH2_H2O, **C6_H2O_H2O},
    symmetries_to_d6={**d6_CH3NH2_H2O, **d6_H2O_H2O},
    mbpol_monomers=[False, True, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

scan_CH3NH2_H2O_H2O_1_Vphys_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_H2O_1, scan_CH3NH2_H2O_H2O_1_Vphys_energies)

In [ ]:
plot_scan(
        scan_CH3NH2_H2O_H2O_1,
        {
                "MP2/aug-cc-pVTZ": scan_CH3NH2_H2O_H2O_1_2b_energies,
                "$V_{phys}$": scan_CH3NH2_H2O_H2O_1_Vphys_2b_energies,
        },
        4,
        -3,
        title="3-body Energies",
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[10].point, system.atoms[0].point)
)

In [ ]:
scan_CH3NH2_H2O_H2O_2, scan_CH3NH2_H2O_H2O_2_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O_H2O, paths.scans.CH3NH2_H2O_H2O.second_energies)
scan_CH3NH2_H2O_H2O_2_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_H2O_2, scan_CH3NH2_H2O_H2O_2_energies)

scan_CH3NH2_H2O_H2O_2_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_CH3NH2_H2O_H2O,
    scan_CH3NH2_H2O_H2O_2,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6={**C6_CH3NH2_H2O, **C6_H2O_H2O},
    symmetries_to_d6={**d6_CH3NH2_H2O, **d6_H2O_H2O},
    mbpol_monomers=[False, True, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

scan_CH3NH2_H2O_H2O_2_Vphys_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_H2O_2, scan_CH3NH2_H2O_H2O_2_Vphys_energies)

In [ ]:
plot_scan(
        scan_CH3NH2_H2O_H2O_2,
        {
                "MP2/aug-cc-pVTZ": scan_CH3NH2_H2O_H2O_2_2b_energies,
                "$V_{phys}$": scan_CH3NH2_H2O_H2O_2_Vphys_2b_energies,
        },
        4,
        -3,
        title="3-body Energies",
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[10].point, system.atoms[0].point)
)

In [ ]:
scan_CH3NH2_H2O_H2O_3, scan_CH3NH2_H2O_H2O_3_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O_H2O, paths.scans.CH3NH2_H2O_H2O.third_energies)
scan_CH3NH2_H2O_H2O_3_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_H2O_3, scan_CH3NH2_H2O_H2O_3_energies)

scan_CH3NH2_H2O_H2O_3_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_CH3NH2_H2O_H2O,
    scan_CH3NH2_H2O_H2O_3,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6={**C6_CH3NH2_H2O, **C6_H2O_H2O},
    symmetries_to_d6={**d6_CH3NH2_H2O, **d6_H2O_H2O},
    mbpol_monomers=[False, True, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

scan_CH3NH2_H2O_H2O_3_Vphys_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_H2O_3, scan_CH3NH2_H2O_H2O_3_Vphys_energies)

In [ ]:
plot_scan(
        scan_CH3NH2_H2O_H2O_3,
        {
                "MP2/aug-cc-pVTZ": scan_CH3NH2_H2O_H2O_3_2b_energies,
                "$V_{phys}$": scan_CH3NH2_H2O_H2O_3_Vphys_2b_energies,
        },
        4,
        -3,
        title="3-body Energies",
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[10].point, system.atoms[0].point)
)

Since we will build polynomials only for 1-, 2-, and 3- body interactions, its worth seeing how $V_{phys}$ performs on these higher-body energies:

In [ ]:
definition_tetramer = read_definition(paths.definitions.systems.tetramer)
clusters_tetramer, clusters_tetramer_energies = read_systems_and_nmer_energies(definition_tetramer, paths.clusters.tetramer_energies)

clusters_tetramer_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_tetramer,
    clusters_tetramer,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6={**C6_CH3NH2_H2O, **C6_H2O_H2O},
    symmetries_to_d6={**d6_CH3NH2_H2O, **d6_H2O_H2O},
    mbpol_monomers=[False, True, True, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

In [ ]:
plot_clusters(
        clusters_tetramer,
        {
            "MP2/aug-cc-pVTZ": calculate_many_body_energies(clusters_tetramer, clusters_tetramer_energies),
            "$V_{phys}$": calculate_many_body_energies(clusters_tetramer, clusters_tetramer_Vphys_energies),
        },
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
        ymin=-9,
        render_scale=0.2,
        title="Cluster 4-body Energies"
)

In [ ]:
definition_pentamer = read_definition(paths.definitions.systems.pentamer)
clusters_pentamer, clusters_pentamer_energies = read_systems_and_nmer_energies(definition_pentamer, paths.clusters.pentamer_energies)

clusters_pentamer_Vphys_energies = calculate_Vphys_nmer_energies(
    definition_pentamer,
    clusters_pentamer,
    symmetry_to_charge=charges_CH3NH2,
    symmetry_to_polarizability={**polarizabilities_CH3NH2, **polarizabilities_H2O},
    symmetry_to_polarizability_factor={**polarizability_factors_CH3NH2, **polarizability_factors_H2O},
    symmetries_to_C6={**C6_CH3NH2_H2O, **C6_H2O_H2O},
    symmetries_to_d6={**d6_CH3NH2_H2O, **d6_H2O_H2O},
    mbpol_monomers=[False, True, True, True, True], # special flag to enable MB-pol geometry-dependent charges for the water molecule,
    aCC=aCC,
    aCD=aCD,
    aDD=aDD,
    aDD12=aDD12,
    aDD13=aDD13,
    aDD14=aDD14,
)

In [ ]:
plot_clusters(
        clusters_pentamer,
        {
            "MP2/aug-cc-pVTZ": calculate_many_body_energies(clusters_pentamer, clusters_pentamer_energies),
            "$V_{phys}$": calculate_many_body_energies(clusters_pentamer, clusters_pentamer_Vphys_energies),
        },
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
        ymin=-9,
        render_scale=0.2,
        title="Cluster 5-body Energies"
)

Of course, there isn't really anything interesting to see here at the 5-body level, since the energies are basically 0.

## 2.2.7 Implementation in MBX

Now, lets put the $V_{phys}$ part of our PEF in MBX.

First, create a fresh copy of MBX (this clones our MBX GitHub repository):

In [ ]:
initialize_mbx(
        paths.MBX
)

Then, we add each part of $V_{phys}$ to MBX:

In [ ]:
add_monomer_type_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        "ch3nh2",
)

In [ ]:
add_excluded_pairs(
        paths.MBX,
        definition_CH3NH2,
        "ch3nh2",
)

In [ ]:
add_charges_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        "ch3nh2",
        symmetry_to_charge=charges_CH3NH2
)

In [ ]:
add_polarizability_factors_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        "ch3nh2",
        symmetry_to_polarizability_factor=polarizability_factors_CH3NH2
)

In [ ]:
add_polarizabilities_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        "ch3nh2",
        symmetry_to_polarizabilities=polarizabilities_CH3NH2
)

In [ ]:
add_reciprocal_space_C6_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        "ch3nh2",
        symmetry_to_reciprocal_space_C6={
                "A": C6_CH3NH2_CH3NH2[("A", "A")] ** (1/2),
                "B": C6_CH3NH2_CH3NH2[("B", "B")] ** (1/2),
                "C": C6_CH3NH2_CH3NH2[("C", "C")] ** (1/2),
                "D": C6_CH3NH2_CH3NH2[("D", "D")] ** (1/2),
        }
)

In [ ]:
add_C6_and_d6_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        definition_H2O,
        "ch3nh2",
        "h2o",
        symmetries_to_C6=C6_CH3NH2_H2O,
        symmetries_to_d6=d6_CH3NH2_H2O
)

In [ ]:
add_C6_and_d6_to_MBX(
        paths.MBX,
        definition_CH3NH2,
        definition_CH3NH2,
        "ch3nh2",
        "ch3nh2",
        symmetries_to_C6=C6_CH3NH2_CH3NH2,
        symmetries_to_d6=d6_CH3NH2_CH3NH2
)

Finally, lets compile MBX. We will compile with optimizations off so that compilation will be fast, but this means that evaluation may be slower later.

In [ ]:
compile_mbx(
        paths.MBX,
        configure_arguments=[
                "CXX=icpc",
                "CC=icc",
                "--disable-optimization"
        ],
        configure=True,
        clean=True,
        num_threads=4
)